In [2]:
import tensorflow as tf
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Import MNIST data
#http://yann.lecun.com/exdb/mnist/
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/", one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [3]:
#TO-DO : reset session
def tf_reset():
    try:
        sess.close()
    except:
        pass
    tf.reset_default_graph()
    return tf.Session()

In [4]:
def get_weights_bias(n_classes):
    w = {
        "w1" : tf.get_variable('W0', shape=(5,5,1,32), initializer=tf.contrib.layers.xavier_initializer()),
        "w2" : tf.get_variable('W1', shape=(5,5,32,64), initializer=tf.contrib.layers.xavier_initializer()),
        'wd1': tf.get_variable('W3', shape=(7*7*64,1024), initializer=tf.contrib.layers.xavier_initializer()), 
        'out': tf.get_variable('W4', shape=(1024,n_classes), initializer=tf.contrib.layers.xavier_initializer())
    }

    b = {
        "b1" : tf.get_variable('B0', shape=(32), initializer=tf.contrib.layers.xavier_initializer()),
        "b2" : tf.get_variable('B1', shape=(64), initializer=tf.contrib.layers.xavier_initializer()),
        "b4" : tf.get_variable('B3', shape=(1024), initializer=tf.contrib.layers.xavier_initializer()),
        "b5" : tf.get_variable('B4', shape=(n_classes), initializer=tf.contrib.layers.xavier_initializer())
    }

    return w,b

def conv2d(X,w,b,strides=1):
    #building conv-layer
    x = tf.nn.conv2d(X,w,strides=[1,strides,strides,1],padding="SAME")
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

def maxpool2d(X, k=2):
    return tf.nn.max_pool(X, ksize=[1, k, k, 1], strides=[1, k, k, 1],padding='SAME')

def fully_connected(X,w,b):
    fc = tf.reshape(X, [-1,w.get_shape().as_list()[0]])
    fc = tf.matmul(fc, w)
    fc = tf.add(fc,b)
    return tf.nn.relu(fc)

def build_model(input_layer,w,b):

    input_layer = tf.reshape(input_layer,[-1,28,28,1])

    #conv-layer-1
    conv1 = conv2d(input_layer,w['w1'],b['b1'])
    conv1 = maxpool2d(conv1, k=2)
    print(conv1.get_shape())

    conv2 = conv2d(conv1,w['w2'],b['b2'])
    conv2 = maxpool2d(conv2, k=2)
    print(conv2.get_shape())

    fc1 = fully_connected(conv2,w['wd1'],b['b4'])
    print(fc1.get_shape())

    out = tf.matmul(fc1,w['out'])

    return tf.add(out,b['b5'])

In [11]:
#TO-DO 
# reset session
sess = tf_reset()

w,b = get_weights_bias(10)

#TO-DO
#Define input_placeholder #(None x 784)
#Define output_placeholder #(None x 10)

input_ph = tf.placeholder(dtype = tf.float32, shape=[None,784])
output_ph = tf.placeholder(dtype=tf.float32, shape=[None,10])

output_pred = build_model(input_ph,w,b)

# Define Loss function # Not reduce Mean for linear regression. Look up lecture 7. Look up AdamOptimizer
# Define optimizer
# Initialize your variables 

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output_pred,labels=output_ph))
opt = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(loss)
#Using Gradient Descent for now. Will try to understand Adam Optimizer before the project. 

pred = tf.nn.softmax(output_pred)
actual = tf.equal(tf.argmax(pred,1),tf.argmax(output_ph,1))
accuracy = tf.reduce_mean(tf.cast(actual,tf.float32))
#I had to google this. 


batch_size = 128
sess.run(tf.global_variables_initializer())

#Begin for loop:
#get slice of input,output using mnist next batch
#run session to compute loss
#print loss w.r.t current iteration value

for i in range(200):
    input_slice,output_slice = mnist.train.next_batch(batch_size)
    _,mse = sess.run([opt,loss],feed_dict={input_ph:input_slice,output_ph:output_slice})
    #This part is from the last lab
    
    if i % 20 == 0:
        mse,acc = sess.run([loss,accuracy],feed_dict={input_ph:input_slice,output_ph:output_slice})
        print("iteration : {}, loss : {:.2f}, acc : {:.2f}".format(i,mse,acc))
        #This code is from lab 2. 
        # We aren't going to be printing all 200 iterations so we will do multiples of 20. 10 prints should be enough.
        

##try it out : compute accuracy using test
#hint : minist.test gives you the test value
# use only a part of the test dataset(slice first 256)



(?, 14, 14, 32)
(?, 7, 7, 64)
(?, 1024)
iteration : 0, loss : 2.27, acc : 0.16
iteration : 20, loss : 1.74, acc : 0.52
iteration : 40, loss : 0.54, acc : 0.86
iteration : 60, loss : 0.26, acc : 0.95
iteration : 80, loss : 0.29, acc : 0.91
iteration : 100, loss : 0.20, acc : 0.95
iteration : 120, loss : 0.11, acc : 0.97
iteration : 140, loss : 0.06, acc : 0.99
iteration : 160, loss : 0.12, acc : 0.98
iteration : 180, loss : 0.17, acc : 0.95
